In [1]:
import pandas as pd
import datetime
import numpy as np
import os
from pyexcelerate import Workbook
import time
import calendar
pd.set_option('display.max_columns', None)

# pd.set_option('display.max_rows', None)

def save_real_csv(datafr,cols_text,path_save):
    df1 = datafr.copy()
    for i in cols_text:
        df1[i] = '='+ '"'+df1[i].astype('str')+'"'
    df1.to_csv(path_save,encoding='utf-8-sig',sep = ';',decimal = ',')   
    df1 = pd.DataFrame()


def rev_to_dict(ind,find):
#     ind = ind.astype('str')комплекс
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt


def save_ex(path,df):
    writer = pd.ExcelWriter(path,engine='openpyxl')
    df.to_excel(writer,sheet_name = 'sheet1')
    writer.save()
    
def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path)
    

    




def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()
    
def get_in_backup(path_backup,name):
    try:
        backup = pd.HDFStore(path_backup)
        ar  = backup[name]
        backup.close()
        return ar
    except:
        backup.close()
        
# превращает цифры строками  в нормальные цифры
def replace_float(df,name_cols):
    try:
        for col_n in name_cols:
            mas = df[col_n].apply(lambda x: x.replace('.',''))
            mas1 = mas.apply(lambda x: x.replace(',','.'))
            df[col_n] =  mas1.astype('float')
    except:
        pass
    
def sklad(x):
    if len(str(x))==3:
        return '0'+str(x)
    elif len(str(x))==2:
        return '00'+str(x)
    else:
        return x
    
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0

In [2]:
folder = '0421 отчет по предцеховым'
directory = 'C:\\Users\\NKedrun\\Desktop\\kazzinc\\Отчет по предцеховым\\'+folder+'\\'
date_report = datetime.datetime(2021,4,1)

In [3]:
exp  = pd.read_excel(directory+'Справочник ЦС к Складу потребности.xlsx',sheet_name='Sheet1')
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})
ost  = pd.read_excel(directory+'Отчет по прицеховым запасам.xlsx',sheet_name='Sheet1')
sprav_mdlv = pd.read_excel(directory+'MDLV.xlsx',sheet_name='Sheet1')
spr_for_obl_ppm = pd.read_excel(directory+'MDLG.xlsx',sheet_name='Sheet1')
sprav_po_kompl = pd.read_excel(directory+'Справочник по комплексам.xlsx',sheet_name='Комплекс',dtype={'Область ППМ':'str','Завод':'str'})

C:\Users\NKedrun\Desktop\kk\NKnotebooks\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7,9,10,11,12,13,16,17,18,19,20,29,30,31,32,37,39,55,58,62,65,66,69,71,75,76,84,85,89,91,92,95,96,98,103,107) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['Дата потребности'] = df['Дата потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]
# собираю дату с группировкой по месяцу
month = df['Дата потребности'].apply(lambda x:  str(x.year)+'-'+str(x.month)+'-01'+ ' 00:00:00')
month =pd.to_datetime(month) 
# заменяю символ - 
df['Количество потребности'] = df['Количество потребности'].str.replace('-','')
df['Баланс (закрытие потребности)'] = df['Баланс (закрытие потребности)'].str.replace('-','')
# Работаю с числами
df['Вид документа потребности'] = df['Вид документа потребности'].astype('str')
df['Вид документа пополнения'] = df['Вид документа пополнения'].astype('str')
# меняем цифры на нормальные флоат
replace_float(df,['Количество потребности','Количество пополнения','Баланс (закрытие потребности)'])
df.fillna(0,inplace=True)
sprav_po_kompl.fillna(0,inplace=True)

In [5]:
df['month'] = df['Дата потребности'].apply(lambda x: str(x.year)+'-'+str(x.month)+ '-01')
df['month'] = pd.to_datetime(df['month']) 

df['Завод'] = df['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Материал'] = df['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Материал'] = df['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

df['Склад потребности'] = df['Склад потребности'].apply(lambda x: str(x).replace('.0','')).astype('str')
df['Склад потребности'] = df['Склад потребности'].apply(lambda x: sklad(x))


spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])


df['Область ППМ']  = df\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')


df['Область ППМ'] = df['Область ППМ'].apply(lambda x: str(x).replace('.0','')).astype('str')

df['Область ППМ'] = df['Область ППМ'].apply(lambda x:"" if x=='0' else x)

df['Завод + Область ППМ'] = df['Завод'].astype('str')+' '+df['Область ППМ'].astype('str')

# чистим
df['Завод + Область ППМ'] =  df['Завод + Область ППМ'].apply(lambda x: str(x).replace('0.0','').replace('.0','')).str.strip()

spr_zav_kompl = rev_to_dict(sprav_po_kompl['Завод + Область ППМ'].astype('str').str.strip(),sprav_po_kompl['Комплекс'])
df['Комплекс'] = df['Завод + Область ППМ'].apply(lambda x: insert_vpr(x,spr_zav_kompl,'Комплекс') )



spr_zav_obl =  rev_to_dict(sprav_mdlv['Область ППМ'].astype('str').str.strip(),sprav_mdlv['Текст ОблПланирПотр'])
df['Имя Завода'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_zav_obl,'Текст ОблПланирПотр')  if x['Область ППМ']=='' else insert_vpr(x['Область ППМ'],spr_zav_obl,'Текст ОблПланирПотр'),axis = 1 )


df['Комплекс'] = df.apply(lambda x:  insert_vpr(x['Завод'],spr_zav_kompl,'Комплекс') if x['Комплекс']==0 else x['Комплекс'],axis = 1)



In [6]:
# df = df.reset_index()

def f(x):
    if x<date_report:
        return 'прошедшая'
    elif x==date_report:
        return 'текущая'
    elif x>date_report:
        return 'будущая'
    
# ost = ost[ost['Кол-во запаса в конце периода']>0]
    
df['month'] = df['month'].apply(lambda x: datetime.datetime(2199,1,1) if x == 0 else x)
    
df['вид потребности'] = df['month'].apply(lambda x:f(x))
df['index'] = [x for x in range(len(df))]
# df = df.set_index(['Имя Завода','Склад потребности','вид потребности'])
df['Имя ЗаводаСклад потребностивид потребности'] = df['Имя Завода'].astype('str')+df['Склад потребности'].astype('str')+df['вид потребности'].astype('str')+df['Материал'].astype('str')
# df = df.set_index(['Имя ЗаводаСклад потребностивид потребности','index'])
df1 = df.groupby('Имя ЗаводаСклад потребностивид потребности')

ost.fillna(0,inplace = True)
exp.fillna(0,inplace = True)
to_str_ost = ['Завод','Склад']

for i in to_str_ost:
    ost[i] = ost[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    ost[i] = ost[i].apply(lambda x: '' if x=='0' else x)
to_str_ost = ['Завод ЦС','Склад']
for i in to_str_ost:
    exp[i] = exp[i].apply(lambda x: str(x).replace('.0','')).astype('str')    
    exp[i] = exp[i].apply(lambda x: '' if x=='0' else x)
    
cols_ = [x for x in exp.columns if str(x).find('Склад потребности')!=-1]
for i in cols_:
    exp[i] = exp[i].apply(lambda x: str(x).replace('.0','')).astype('str')    
    exp[i] = exp[i].apply(lambda x: '' if x=='0' else x)

In [7]:
start_time = time.time()
df['Количество потребности1'] = df['Количество потребности']
for ind in ost.index:
    if ind%1000 ==0:
        print(ind)
    dat = exp[(exp['Завод ЦС'].values == ost.at[ind,'Завод'])&(exp['Склад'].values == ost.at[ind,'Склад'])]
    
    if dat.shape[0]>0:
#         print(dat.shape)
        for ind1 in dat.index:
            for col in cols_:
                if dat.at[ind1,col]!='':
                    
                    try:
#                         
                        dat1_pr = df1.get_group(dat.at[ind1,'Наименование области ППМ']+dat.at[ind1,col]+'прошедшая'+ost.at[ind,'Материал'])
                        kompleks = dat1_pr['Комплекс'].unique()[0]
                        if len([x for x in ost.columns if str(kompleks)+'/'+'прошедшая'==x])==0:
                            ost[str(kompleks)+'/'+'прошедшая'] = 0
                            
                        ost.loc[ind,   str(kompleks)+'/'+'прошедшая'] = ost.loc[ind,   str(kompleks)+'/'+'прошедшая'] + dat1_pr['Количество потребности'].sum()
                        
                        df.loc[dat1_pr.index,'Количество потребности'] = 0
                        
                    except KeyError:
                        pass
                    
                    try:
#                         
                        dat1_curr = df1.get_group(dat.at[ind1,'Наименование области ППМ']+dat.at[ind1,col]+'текущая'+ost.at[ind,'Материал'])
                        kompleks = dat1_curr['Комплекс'].unique()[0]
                        if len([x for x in ost.columns if str(kompleks)+'/'+'текущая'==x])==0:
                            ost[str(kompleks)+'/'+'текущая'] = 0
                            
                        ost.loc[ind,   str(kompleks)+'/'+'текущая'] = ost.loc[ind,   str(kompleks)+'/'+'текущая'] + dat1_curr['Количество потребности'].sum()
                        df.loc[dat1_curr.index,'Количество потребности'] = 0
                    except KeyError:
                        pass
                    
                    try:
#                         dat1_bud = df.xs([dat.at[ind1,'Наименование области ППМ']+dat.at[ind1,col]+'будущая'+ost.at[ind,'Материал']],drop_level=False)

                        dat1_bud = df1.get_group(dat.at[ind1,'Наименование области ППМ']+dat.at[ind1,col]+'будущая'+ost.at[ind,'Материал'])
                        
                        kompleks = dat1_bud['Комплекс'].unique()[0]
                        if len([x for x in ost.columns if str(kompleks)+'/'+'будущая'==x])==0:
                            ost[str(kompleks)+'/'+'будущая'] = 0
                        ost.loc[ind,   str(kompleks)+'/'+'будущая'] = ost.loc[ind,   str(kompleks)+'/'+'будущая'] + dat1_bud['Количество потребности'].sum()
                        df.loc[dat1_bud.index,'Количество потребности'] = 0
                    except KeyError:
                        pass
                        
                    
                    

print("--- %s seconds ---" % (time.time() - start_time)) 

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
--- 1141.6545932292938 seconds ---


In [ ]:
ost

In [ ]:
df

In [ ]:
ost.loc[:,'РГОК/текущая':].sum().sum()

In [ ]:
df['Количество потребности1'].sum()-df['Количество потребности'].sum()

In [ ]:
ost

In [ ]:
df = get_in_backup(path_backup='backup.h5',name='extract')

In [ ]:
df_val = df.values

In [ ]:
df_val[:,0]==

In [ ]:
df_val

In [ ]:
ost.loc[:,'РГОК/текущая':].sum().sum()

In [ ]:
ost

In [ ]:
df